# BACKTESTING STOCK MARKET SHORT PREDICTION MODEL

@author: Matheus José Oliveira dos Santos

## 1. Imports

In [1]:
import sys
import os
from datetime import date, datetime, timedelta

import pandas as pd

sys.path.append(os.getcwd()+'\\..\\src')
import db_interface
import portfolio_backtest

## 2. class inheritance

In [2]:
class Backtest(portfolio_backtest.Backtest):
    def get_data(self) -> pd.DataFrame:
        print("getting data")
        _ref_date = self._date_indicator
        table_name = "financials"
        
        my_db = db_interface.DB_interface("FINANCE_DB")

        with my_db:
            for i in range(5):
                #query only brazilian stocks
                query = "SELECT * FROM {0} WHERE date = '{1}'".format(table_name, _ref_date.strftime("%Y-%m-%d"))
                df_data = my_db.read_by_command(query)
                if df_data.shape[0] == 0:
                    #if there are no data, get the day before
                    _ref_date = _ref_date - timedelta(days=1)
                else:
                    break #exit loop
                    raise ValueError("There are not enough data")
        
        #pre-processing
        df_sample = df_data.copy.iloc[:,2:-3]
        sc = joblib.load(self._sc)
        df_sample_scaled = sc.transform(df_sample)
        
        #add first column as BBGTicker
        df_sample_scaled.insert(0, 'BBGTicker', df_data['BBGTicker'])
        
        return df_sample_scaled
    
    def new_year(self) -> None:
        pass

## 3. Backtest

In [3]:
start_date = date(2022,1,1)
end_date = date(2023,8,1)
delta = timedelta(weeks=1)

bt = Backtest(start_date, end_date, delta)
bt.set_date_indicator(date(2023,7,21))
df = bt.get_data()
#bt.executar()

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
df = bt.get_performance()